In [49]:
#Imports for model
from os import path
import numpy as np
import pandas as pd
import os
import csv
import re
#Install import_ipynb to import other notebooks
# !pip install import_ipynb
import import_ipynb
import tensorflow as tf
# physical_devices = tf.config.list_physical_devices('GPU') 
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow import keras

In [50]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
#Import Glove class
from glove import Glove

In [4]:
#Create instance of glove
glove = Glove()

Found 1917494 word vectors.


In [51]:
glove.vector("__OOV__")

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [73]:
#Clean sentence (remove non alpha chars)
p = re.compile(r'<.*?>')
def cleanSentence(sentence, html = False):
    if html: sentence = p.sub('', sentence) 
    sentence = ''.join([i.lower() if i.isalpha() else " " if (i==" " or i=="-" or i=="_") else "" for i in sentence])
    return sentence

#Iterator for given file
def iterate_file(file_path):
    with open(file_path, "r", encoding = 'utf8') as f:
        reader = csv.DictReader(f, delimiter=",")
        for row in reader:
            yield row

def convert_strlist_to_vec(strlist, max_sequence_length, vec_dim):
    vec = np.zeros((max_sequence_length, vec_dim))
    for i in range(max_sequence_length):
        if i == len(strlist):
            break
        vec[i] = glove.vector(strlist[i])
    return vec

# def data_generator(file_path, minibatch_size=5000, max_sequence_length=15, vec_dim=300, Infinite_loop=True):
#     i = 0
#     X = []
#     Y = []
#     while True:
#         for row in iterate_file(file_path):
#             info = row["tags"].split("|") + row["title"].split()
#             xi = convert_strlist_to_vec(info, max_sequence_length, vec_dim)
#             yi =  (float(row["stars"])>3)
#             X.append(xi)
#             Y.append(yi)
#             i +=1
#             if i>=minibatch_size:
#                 yield np.array(X), np.array(Y)#, [None]
#                 i = 0
#                 X = []
#                 Y = []
#         if not Infinite_loop: break

def data_generator(file_path, minibatch_size=5000, max_sequence_length=15, vec_dim=300, Infinite_loop=True):
    i = 0
    X1 = []
    X2= []
    Y = []
    while True:
        for row in iterate_file(file_path):
            #----------------
            #info = row["tags"].split("|") + ["."] +row["title"].split() + ["."] #+ row["body"].split()
            info = row["title"].split()
            x1i = convert_strlist_to_vec(info, max_sequence_length, vec_dim)
            #----------------
            date = row["creation_date"]
            x2i = [0]*10
            x2i[int(date[3])] = 1
#             print(date, x2i)
            #----------------
            yi =  float(float(row["stars"])>3)
            #----------------
            X1.append(x1i)
            X2.append(x2i)
            Y.append(yi)
            i +=1
            if i>=minibatch_size:
                yield [np.array(X1), np.array(X2)], np.array(Y)#, [None]
                i = 0
                X1 = []
                X2 = []
                Y = []
        if not Infinite_loop: break

# def make_gen_callable(_gen):
#         def gen():
#             for x,y in _gen:
#                  yield x,y
#         return gen
        
# def create_dataset(file_path, minibatch_size=3000, max_sequence_length=15):
#     generator = make_gen_callable(data_generator(file_path, minibatch_size, max_sequence_length))
#     dataset = tf.data.Dataset.from_generator(generator=generator,
#                                              output_types=(tf.float32,tf.bool),
#                                              output_shapes = ((None,max_sequence_length,300), (None,))
#                                             )
#     return dataset
        
# #Parameters
MAX_SEQUENCE_LENGTH = 30
VEC_DIM = 300
INPUT_FILE_TRAIN = "../processed_files/data_stack_shuffled_heavy_train.csv"
INPUT_FILE_VAL = "../processed_files/data_stack_shuffled_heavy_val.csv"
INPUT_FILE_TEST = "../processed_files/data_stackoverflow_heavy_test.csv"

train_dataset_size = 604695  #<<<---------------OJO!! xoxo
val_dataset_size = 18051

minibatch_size = 512
train_generator = data_generator(INPUT_FILE_TRAIN, minibatch_size, MAX_SEQUENCE_LENGTH)
val_generator = data_generator(INPUT_FILE_VAL, minibatch_size, MAX_SEQUENCE_LENGTH)
# train_generator = create_dataset(INPUT_FILE_TRAIN, minibatch_size, MAX_SEQUENCE_LENGTH)
# val_generator = create_dataset(INPUT_FILE_VAL, minibatch_size, MAX_SEQUENCE_LENGTH)


# cont = 0
# for data in data_generator(INPUT_FILE_VAL):
#     data
#     cont +=1
#     if cont >= 10:
#         break

        

In [74]:
# Create model #1
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Input, LSTM, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.layers import Bidirectional, Dropout, SpatialDropout1D, Concatenate
from tensorflow.keras import Model

# inputs: A 3D tensor with shape [batch, timesteps, feature].
# inputs = tf.random.normal([32, 10, 8])

# X = SpatialDropout1D(0.4)(Input)
# x = LSTM(64, activation="relu", dropout=0.2, recurrent_dropout=0.2)(x)
# x = Dropout(0.2)(x)
#----------------------------------------------------------------------
input1 = Input(shape=(MAX_SEQUENCE_LENGTH,300))
input2 = Input(shape= (10,))
x = Bidirectional(LSTM(256, activation="relu", return_sequences=True))(input1)
x = Bidirectional(LSTM(256, activation="relu"))(x)
x = Flatten()(x)
x = Concatenate()([x, input2])
x = Dense(256 ,activation="relu")(x)
x = Dense(128 ,activation="relu")(x)
# preds = Dense(1)(x)
preds = Dense(1, activation="sigmoid")(x)

model1 = Model([input1, input2], preds)
model1.compile(loss = keras.losses.binary_crossentropy,
              # loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(0.0003),# 0.0003
              # metrics=["mean_absolute_error"])
              metrics=["accuracy"])
model1.summary()


Model: "functional_39"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_39 (InputLayer)           [(None, 30, 300)]    0                                            
__________________________________________________________________________________________________
bidirectional_37 (Bidirectional (None, 30, 512)      1140736     input_39[0][0]                   
__________________________________________________________________________________________________
bidirectional_38 (Bidirectional (None, 512)          1574912     bidirectional_37[0][0]           
__________________________________________________________________________________________________
flatten_19 (Flatten)            (None, 512)          0           bidirectional_38[0][0]           
______________________________________________________________________________________

In [75]:
#Fit model
# model1.fit(input_sequences_train, labels_train, validation_data=(input_sequences_test, labels_test), epochs=50, batch_size=3000)
history = model1.fit( 
            train_generator,
            validation_data = val_generator,
            steps_per_epoch = train_dataset_size//minibatch_size,
            validation_steps = val_dataset_size//minibatch_size,
            epochs=20,
            use_multiprocessing=False)
# model1.fit_generator(train_dataset, validation_data=test_dataset, epochs=1)

Epoch 1/20
1181/1181 [==============================] - 779s 659ms/step - loss: 0.6659 - accuracy: 0.5947 - val_loss: 0.6638 - val_accuracy: 0.6008
Epoch 2/20
1181/1181 [==============================] - 785s 664ms/step - loss: 0.6596 - accuracy: 0.6057 - val_loss: 0.6618 - val_accuracy: 0.6049
Epoch 3/20
1181/1181 [==============================] - 809s 685ms/step - loss: 0.6573 - accuracy: 0.6091 - val_loss: 0.6609 - val_accuracy: 0.6054
Epoch 4/20
1181/1181 [==============================] - 481s 408ms/step - loss: 0.6565 - accuracy: 0.6099 - val_loss: 0.6615 - val_accuracy: 0.6056
Epoch 5/20
1181/1181 [==============================] - 479s 406ms/step - loss: 0.6548 - accuracy: 0.6129 - val_loss: 0.6606 - val_accuracy: 0.6074
Epoch 6/20
1181/1181 [==============================] - 474s 402ms/step - loss: 0.6546 - accuracy: 0.6133 - val_loss: 0.6583 - val_accuracy: 0.6096
Epoch 7/20
1181/1181 [==============================] - 478s 405ms/step - loss: 0.6533 - accuracy: 0.6150 - val_

In [92]:
questions = [
              "compressing / decompressing folders & files",
              "HOW TO decompress and compress files and folders",
              "how to load a specific version of an assembly",
              "how would one code test and set behavior without a special hardware instruction?",
             "can you debug a .net app with only the source code of one file?",
             "what columns generally make good indexes?",
             "why is there no generic synchronized queue in .net?"
]

for question in questions:
    input_sample = convert_strlist_to_vec(question.split(), MAX_SEQUENCE_LENGTH, VEC_DIM)
    input_sample = input_sample[np.newaxis,...]
    year = np.array([[0,0,0,0,0,0,0,0,0,1]])
    print(year.shape)
    print(model1.predict([input_sample, year]))



(1, 10)
[[0.33768833]]
(1, 10)
[[0.4795051]]
(1, 10)
[[0.38035446]]
(1, 10)
[[0.41057754]]
(1, 10)
[[0.5227095]]
(1, 10)
[[0.2439075]]
(1, 10)
[[0.5483649]]


In [82]:
predictions = model1.predict(input_sequences_test)
for y_hat,y  in zip(predictions[:50], labels_test[:50]):
    print(y,":", y_hat)

NameError: name 'input_sequences_test' is not defined

In [ ]:
#create word to index dictionary: word->index
# index 0 is for unnexistent word
word_index = {}
cont = 0
for word in glove.embeddings.keys():
    cont +=1
    word_index[word] = cont

#Processing of GloVe data into the embedding matrix to use in Keras
embedding_matrix = np.zeros((len(word_index) + 1, glove.dim))
EMBEDDING_DIM = glove.dim

for word, i in word_index.items():
    embedding_matrix[i] = glove.vector(word)

# delete glove dictionary to save memory RAM
#del glove

In [ ]:
# Create model #1
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

#del embedding_matrix

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(16, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(16, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(16, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(1, activation='relu')(x)

model1 = Model(sequence_input, preds)
model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

# happy learning!
# model1.fit(x_train, y_train, validation_data=(x_val, y_val),
#           epochs=2, batch_size=128)


In [77]:
#Export Glove Model
import pickle
with open("../processed_files/glove_model.pkl", "wb") as f:
    pickle.dump(glove, f)

In [79]:
#Export NN model
model1.save("model.h5")